### Imports

In [8]:
from sqlalchemy import create_engine
import pandas as pd

### Create Database Engine

In [9]:
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

### Test a Query

In [13]:
con = engine.connect()
rs = con.execute("SELECT * FROM Facilities")
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
con.close()

In [15]:
df

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


#### Test with Context Manager

In [17]:
with engine.connect() as con:
    rs = con.execute("SELECT * FROM Facilities")
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys() 

In [18]:
df

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


## Questions

**Q10:** </br>
Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [30]:
with engine.connect() as con:
    rs = con.execute("SELECT f.name, SUM(CASE WHEN b.memid = 0 THEN (b.slots * f.guestcost) \
                            ELSE (b.slots * f.membercost) END) AS revenue \
                    FROM Bookings AS b \
                    INNER JOIN Facilities AS f \
                        ON b.facid = f.facid \
                    INNER JOIN Members AS m \
                        ON b.memid = m.memid \
                    GROUP BY f.name \
                    HAVING SUM(CASE WHEN b.memid = 0 THEN (b.slots * f.guestcost) ELSE (b.slots * f.membercost) END) < 1000 \
                    ORDER BY revenue;")
    rev_df = pd.DataFrame(rs.fetchall())
    rev_df.columns = rs.keys() 

In [31]:
rev_df

,name,revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


**Q11:** </br>
Produce a report of members and who recommended them in alphabetic surname,firstname order

In [41]:
with engine.connect() as con:
    rs = con.execute("SELECT DISTINCT (m1.surname || ', ' || m1.firstname) AS member_name, (m2.surname || ', ' || m2.firstname) AS reccomended_by \
                    FROM Members AS m1 \
                    INNER JOIN Members AS m2 \
                        ON m1.recommendedby = m2.memid \
                    ORDER BY 1,2;")
    refer_df = pd.DataFrame(rs.fetchall())
    refer_df.columns = rs.keys() 

In [42]:
refer_df

,member_name,reccomended_by
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"
5,"Coplin, Joan","Baker, Timothy"
6,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
8,"Genting, Matthew","Butters, Gerald"
9,"Hunt, John","Purview, Millicent"


**Q12:** </br>
Find the facilities with their usage by member, but not guests

In [52]:
with engine.connect() as con:
    rs = con.execute("SELECT f.name AS facility, (m.surname || ', ' || m.firstname) AS member_name, SUM(b.slots) AS slots_utilized \
                    FROM Bookings AS b \
                    INNER JOIN Facilities AS f \
                        ON b.facid = f.facid \
                    INNER JOIN Members AS m \
                        ON b.memid = m.memid \
                    WHERE b.memid <> 0 \
                    GROUP BY f.name, m.surname, m.firstname \
                    ORDER BY slots_utilized DESC, facility, member_name;")
    usage_df = pd.DataFrame(rs.fetchall())
    usage_df.columns = rs.keys() 

In [53]:
usage_df

,facility,member_name,slots_utilized
0,Badminton Court,"Smith, Darren",432
1,Pool Table,"Rownam, Tim",282
2,Massage Room 1,"Rownam, Tim",176
3,Tennis Court 1,"Butters, Gerald",171
4,Tennis Court 2,"Boothe, Tim",168
...,...,...,...
197,Table Tennis,"Baker, Anne",2
198,Table Tennis,"Butters, Gerald",2
199,Table Tennis,"Hunt, John",2
200,Pool Table,"Farrell, Jemima",1


**Q13:** </br>
Find the facilities usage by month, but not guests

In [ ]:
with engine.connect() as con:
    rs = con.execute("SELECT f.name AS facility, (m.surname || ', ' || m.firstname) AS member_name, SUM(b.slots) AS slots_utilized \
                    FROM Bookings AS b \
                    INNER JOIN Facilities AS f \
                        ON b.facid = f.facid \
                    INNER JOIN Members AS m \
                        ON b.memid = m.memid \
                    WHERE b.memid <> 0 \
                    GROUP BY f.name, m.surname, m.firstname \
                    ORDER BY slots_utilized DESC, facility, member_name;")
    monthly_df = pd.DataFrame(rs.fetchall())
    monthly_df.columns = rs.keys() 